## Business Understanding

### Zielsetzung
Das Ziel dieses Projekts ist es, fünf potenzielle Städte für einen sechsmonatigen Aufenthalt als Remote Worker in der Kreativbranche zu identifizieren. Der Fokus liegt dabei auf tropisch warmen Ländern, da diese nicht nur ein angenehmes Klima bieten, sondern auch einzigartige wirtschaftliche und infrastrukturelle Eigenschaften aufweisen.

### Ausgangssituation und Fragestellungen
Wir arbeiten mit dem globalen „Cost of Living“-Datensatz von Kaggle, der uns detaillierte Informationen über Lebenshaltungskosten in Städten weltweit liefert. Anhand dieses Datensatzes wollen wir untersuchen, wie sich Lebenshaltungskosten und Infrastruktur in tropisch warmen Ländern im Vergleich zu Regionen wie Europa und Nordamerika darstellen.

### Zentrale Hypothesen
1. **Tropisch Warme Länder sind meist Touristen Hotspots:**  
   Diese Hypothese basiert auf der Annahme, dass der touristische Reiz tropischer Länder zu spezifischen Preisstrukturen führt. Saisonale Schwankungen und hohe Nachfrage in bestimmten Sektoren (z.B. Unterkunft, Gastronomie) könnten die Lebenshaltungskosten in diesen Städten beeinflussen.

2. **Tropische warme Länder haben tendenziell niedrigere Kosten als Europa und Nordamerika:**  
   Aufgrund unterschiedlicher wirtschaftlicher Rahmenbedingungen und oft geringerer Durchschnittseinkommen gehen wir davon aus, dass die allgemeinen Lebenshaltungskosten in tropisch warmen Ländern unter denen in entwickelten Regionen liegen.

3. **Eine gute Infrastruktur korreliert mit höheren Kosten:**  
   Es wird vermutet, dass Städte mit moderner Infrastruktur, etwa in Bereichen wie Internet und öffentlichem Verkehr, höhere Lebenshaltungskosten aufweisen. Diese Investitionen in Infrastruktur könnten sowohl den Komfort als auch das Preisniveau in die Höhe treiben.

### Weiteres Vorgehen
Auf Basis dieser Hypothesen werden wir in der Datenexploration und -analyse prüfen, inwiefern diese Zusammenhänge bestehen. Die gewonnenen Erkenntnisse sollen als Entscheidungsgrundlage dienen, um letztlich fünf Städte auszuwählen, die sowohl kosteneffizient als auch attraktiv hinsichtlich Klima, Infrastruktur und touristischem Potenzial sind.

In [72]:
# Aufgabe 2 Data Preperation (Mappen der Kontinente und löschen der schlechten Qualität)
import pandas as pd
import requests
from thefuzz import process

def find_best_match(country):
    # Versuche zuerst ein direktes Mapping
    if country in country_to_continent:
        return country_to_continent[country]

    # Versuche es mit dem ersten Wort
    first_word = country.split()[0]
    if first_word in country_to_continent:
        return country_to_continent[first_word]

    # Falls kein direkter Treffer: Fuzzy Matching versuchen
    best_match, score = process.extractOne(country, country_to_continent.keys())
    if score > 80:  # 80 bedeutet "ziemlich sicher"
        return country_to_continent[best_match]

    # Falls nichts gefunden, None zurückgeben
    return None

# Entferne die Spalten mit schlechter Qualität
data = pd.read_csv("./data/cost-of-living_v2.csv")
data.drop(data[data['data_quality'] == 0].index, inplace=True)
data.sort_values(by=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)


# Mappen der Länder zu Kontinenten
url = "https://www.apicountries.com/countries"
response = requests.get(url)
countries = response.json()

country_to_continent = {}
for item in countries:
    country_to_continent[item["name"]] = item["region"]

    # Alternative Namen hinzufügen, falls vorhanden
    if "altSpellings" in item:
        for alt_name in item["altSpellings"]:
            country_to_continent[alt_name] = item["region"]

data["continent"] = data["country"].apply(find_best_match)
#countries = list(filter(lambda item: item['region'] != 'Europe', countries))
